# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2022 3:11PM,239495,10,PRF-30967,Bio-PRF,Released
1,Jun 13 2022 3:11PM,239495,10,PRF-31001,Bio-PRF,Released
2,Jun 13 2022 3:11PM,239495,10,PRF-31039,Bio-PRF,Released
3,Jun 13 2022 3:11PM,239495,10,PRF-31057,Bio-PRF,Released
4,Jun 13 2022 3:11PM,239495,10,PRF-31169,Bio-PRF,Released
5,Jun 13 2022 3:11PM,239495,10,PRF-31175,Bio-PRF,Released
6,Jun 13 2022 3:11PM,239495,10,PRF-31222,Bio-PRF,Released
7,Jun 13 2022 3:11PM,239495,10,PRF-31259,Bio-PRF,Released
8,Jun 13 2022 3:11PM,239495,10,PRF-31290,Bio-PRF,Released
9,Jun 13 2022 3:11PM,239495,10,PRF-31293,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,239491,Released,27
44,239492,Released,1
45,239493,Released,3
46,239494,Released,1
47,239495,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239491,NaN,NaN,27.0
239492,NaN,NaN,1.0
239493,NaN,NaN,3.0
239494,NaN,NaN,1.0
239495,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239388,0.0,0.0,1.0
239389,0.0,0.0,1.0
239398,0.0,8.0,6.0
239407,0.0,1.0,0.0
239428,0.0,0.0,40.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239388,0,0,1
239389,0,0,1
239398,0,8,6
239407,0,1,0
239428,0,0,40


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239388,0,0,1
1,239389,0,0,1
2,239398,0,8,6
3,239407,0,1,0
4,239428,0,0,40


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239388,,,1
1,239389,,,1
2,239398,,8,6
3,239407,,1,
4,239428,,,40


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2022 3:11PM,239495,10,Bio-PRF
10,Jun 13 2022 2:59PM,239494,21,"NBTY Global, Inc."
11,Jun 13 2022 2:54PM,239493,16,Sartorius Bioprocess Solutions
14,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc."
15,Jun 13 2022 2:51PM,239491,10,"Methapharm, Inc."
42,Jun 13 2022 2:49PM,239490,19,"NAPP Technologies, LLC"
43,Jun 13 2022 2:48PM,239489,10,"Methapharm, Inc."
52,Jun 13 2022 2:40PM,239479,10,ISDIN Corporation
85,Jun 13 2022 2:36PM,239485,21,"NBTY Global, Inc."
86,Jun 13 2022 2:21PM,239484,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2022 3:11PM,239495,10,Bio-PRF,,,10
1,Jun 13 2022 2:59PM,239494,21,"NBTY Global, Inc.",,,1
2,Jun 13 2022 2:54PM,239493,16,Sartorius Bioprocess Solutions,,,3
3,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc.",,,1
4,Jun 13 2022 2:51PM,239491,10,"Methapharm, Inc.",,,27
5,Jun 13 2022 2:49PM,239490,19,"NAPP Technologies, LLC",,,1
6,Jun 13 2022 2:48PM,239489,10,"Methapharm, Inc.",,,9
7,Jun 13 2022 2:40PM,239479,10,ISDIN Corporation,,,33
8,Jun 13 2022 2:36PM,239485,21,"NBTY Global, Inc.",,,1
9,Jun 13 2022 2:21PM,239484,10,MedStone Pharma LLC,,,57


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 3:11PM,239495,10,Bio-PRF,10,,
1,Jun 13 2022 2:59PM,239494,21,"NBTY Global, Inc.",1,,
2,Jun 13 2022 2:54PM,239493,16,Sartorius Bioprocess Solutions,3,,
3,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc.",1,,
4,Jun 13 2022 2:51PM,239491,10,"Methapharm, Inc.",27,,
5,Jun 13 2022 2:49PM,239490,19,"NAPP Technologies, LLC",1,,
6,Jun 13 2022 2:48PM,239489,10,"Methapharm, Inc.",9,,
7,Jun 13 2022 2:40PM,239479,10,ISDIN Corporation,33,,
8,Jun 13 2022 2:36PM,239485,21,"NBTY Global, Inc.",1,,
9,Jun 13 2022 2:21PM,239484,10,MedStone Pharma LLC,57,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 3:11PM,239495,10,Bio-PRF,10,,
1,Jun 13 2022 2:59PM,239494,21,"NBTY Global, Inc.",1,,
2,Jun 13 2022 2:54PM,239493,16,Sartorius Bioprocess Solutions,3,,
3,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc.",1,,
4,Jun 13 2022 2:51PM,239491,10,"Methapharm, Inc.",27,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 3:11PM,239495,10,Bio-PRF,10.0,NaN,NaN
1,Jun 13 2022 2:59PM,239494,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jun 13 2022 2:54PM,239493,16,Sartorius Bioprocess Solutions,3.0,NaN,NaN
3,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jun 13 2022 2:51PM,239491,10,"Methapharm, Inc.",27.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2022 3:11PM,239495,10,Bio-PRF,10.0,0.0,0.0
1,Jun 13 2022 2:59PM,239494,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jun 13 2022 2:54PM,239493,16,Sartorius Bioprocess Solutions,3.0,0.0,0.0
3,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jun 13 2022 2:51PM,239491,10,"Methapharm, Inc.",27.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2873637,292.0,0.0,1.0
12,478938,1.0,1.0,0.0
15,1197211,44.0,5.0,59.0
16,478900,3.0,1.0,0.0
18,239477,1.0,0.0,0.0
19,957913,23.0,0.0,0.0
20,957797,57.0,37.0,0.0
21,2155298,8.0,1.0,0.0
22,478777,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2873637,292.0,0.0,1.0
1,12,478938,1.0,1.0,0.0
2,15,1197211,44.0,5.0,59.0
3,16,478900,3.0,1.0,0.0
4,18,239477,1.0,0.0,0.0
5,19,957913,23.0,0.0,0.0
6,20,957797,57.0,37.0,0.0
7,21,2155298,8.0,1.0,0.0
8,22,478777,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,292.0,0.0,1.0
1,12,1.0,1.0,0.0
2,15,44.0,5.0,59.0
3,16,3.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,23.0,0.0,0.0
6,20,57.0,37.0,0.0
7,21,8.0,1.0,0.0
8,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,292.0
1,12,Released,1.0
2,15,Released,44.0
3,16,Released,3.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,1.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,1.0,5.0,1.0,0.0,0.0,37.0,1.0,0.0
Released,292.0,1.0,44.0,3.0,1.0,23.0,57.0,8.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,1.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,5.0,1.0,0.0,0.0,37.0,1.0,0.0
2,Released,292.0,1.0,44.0,3.0,1.0,23.0,57.0,8.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,1.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,5.0,1.0,0.0,0.0,37.0,1.0,0.0
2,Released,292.0,1.0,44.0,3.0,1.0,23.0,57.0,8.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()